In [1]:
# import required libraries

import pandas as pd
import numpy as np
import requests as rq
import json
import logging
import getpass
import os

## Step 1 : Take credentials from user

In [ ]:
uName = input("Enter your UserName:\n")
print("Enter Your Password")
pwd = getpass.getpass()

## Step 2 : Getting Session ID

### Defining URL, Body & Headers

In [3]:
#Reading URL from text file
file= open('./credentials.txt')

content = file.read()
#print(content)
file.close()

credential_data = dict(subString.split("=") for subString in content.split(";"))

#Defining URL
auth_URL = f"{credential_data['vaultDNS']}api{credential_data['version_Api']}/auth"

#Defining Body
auth_body = {
    'username': uName,
    'password': pwd
}

#Defining Header
auth_headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'application/json'
}

### Get the Session ID

In [4]:
auth_response = rq.post(auth_URL, headers = auth_headers, data = auth_body)


auth_response_data = json.loads(auth_response.content)

sessionId=''

if auth_response_data['responseStatus'] == 'SUCCESS':
    sessionId = auth_response_data['sessionId']
    print('sessionId stored in "sessionId variable"')
    print('Response Status Below :')
    print(auth_response_data['responseStatus'])
    #print('sessionId Below :')
    #print(sessionId)
else:
    print('something went wrong')
    print(auth_response_data['responseStatus'])
    print(auth_response_data['responseMessage'])
   

sessionId stored in "sessionId variable"
Response Status Below :
SUCCESS


## STEP 3 : Downloading the Viewable Rendition for Document(s)

#### import the Object Loader file or DataFrame

In [ ]:
docID_loader_file = pd.read_csv('yourFilecontainingDocIDs.csv')
docID_loader_file.head()

In [6]:
docID_loader_file.shape

(17, 18)

In [7]:
# Defing headers for all attachment files GET response

docID_VR_header = {
    'Authorization': sessionId,
    'Accept': 'application/json'
}

In [ ]:
for i in docID_loader_file.id:
    
    # Defining document viewable rendition URL Path
    docID_VR_url = f"{credential_data['vaultDNS']}api{credential_data['version_Api']}/objects/documents/{i}/renditions/viewable_rendition__v"
    #print(docID_VR_url)   #This will print all URl for Object IDs
    
    #Sending the HTTP Request
    docID_VR_response = rq.get(docID_VR_url, headers = docID_VR_header)
    
    #print('########')
    #print(docID_VR_response.headers) #This will print the status code
    #print('########')

    docID_VR_headers_string_convert = json.dumps(dict(docID_VR_response.headers))
    #print(docID_VR_headers_string_convert)

    docID_VR_headers_dict_convert = json.loads(docID_VR_headers_string_convert)
    
    print(f"Remaining API Burst Limit {docID_VR_headers_dict_convert['X-VaultAPI-BurstLimitRemaining']}")
    #print(docID_VR_headers_dict_convert)
    if 'Content-Disposition'in docID_VR_headers_dict_convert:
        print("#######################################")
        print('content-disposition found')
        if 'filename' in docID_VR_headers_dict_convert['Content-Disposition']: 
                    print('Viewable Rendition available for docId: ' + str(i))         

                    #Naming the Filename
                    filename = docID_VR_headers_dict_convert['Content-Disposition'].split("''")[1]
                    filename_id = f"docID_{i}_{filename}".replace('%20', ' ')                 
                    filename_id = filename_id.replace('%2C', ',')
                    filename_id = filename_id.replace('%28', '(')
                    filename_id = filename_id.replace('%29', ')')
                    filename_id = filename_id.replace('%C2%B0','°')
                    filename_id = filename_id.replace('%E2%80%93','-')
                    filename_id = filename_id.replace('%C2%AE','®')
                    filename_id = filename_id.replace('%E2%84%A2','™')
                    filename_id = filename_id.replace('%CE%B1','α')
                    filename_id = filename_id.replace('%CE%B2','β')
                    filename_id = filename_id.replace('%CE%B3','γ')
                    filename_id = filename_id.replace('%CE%B4','δ')
                    filename_id = filename_id.replace('%25','%')
                    filename_id = filename_id.replace('%E2%80%99',"'")
                    filename_id = filename_id.replace('%27',"'")
        
                    #print(filename_id)


                    #Create a logFile
                    logger = logging.getLogger(__name__)
                    logging.basicConfig(filename="logFile.log",
                                encoding="utf-8",
                                level=logging.DEBUG,
                                format='%(asctime)s %(message)s', 
                                datefmt='%m/%d/%Y %I:%M:%S %p')

                    logger.info('SUCCESS :: Doc Id ::' + str(i) + ' - FOUND viewable rendition')

                    #Download the filename to local
                    directory_path = r'PATH WHERE YOU WANT TO SAVE THE VIEWABLE RENDITIONS'
                    file_path = os.path.join(directory_path, filename_id)
                    with open(file_path, 'wb') as output_file:
                            output_file.write(docID_VR_response.content)
                            print('Viewable Rendition downloaded for docID:' + str(i))
                            print("#######################################")

        else:
                    print("filename not found for docId: " + str(i))
                    print("#######################################")
                    logger = logging.getLogger(__name__)
                    logging.basicConfig(filename="logFile.log",
                                encoding="utf-8",
                                level=logging.DEBUG,
                                format='%(asctime)s %(message)s', 
                                datefmt='%m/%d/%Y %I:%M:%S %p')
                    logger.info('WARNING Content-Disposition(Available):: Doc Id ::' + str(i) + ' has no viewable rendition or something went wrong, Check Manually')
     
    else:  
        print("Content-Disposition not found for docId: " + str(i))
        logger = logging.getLogger(__name__)
        logging.basicConfig(filename="logFile.log",
                            encoding="utf-8",
                            level=logging.DEBUG,
                            format='%(asctime)s %(message)s', 
                            datefmt='%m/%d/%Y %I:%M:%S %p')
        logger.info('WARNING Content-Disposition(NA) :: Doc Id ::' + str(i) + ' has no viewable rendition or something went wrong, Check Manually')


print("############################")
print("############################")
print("TASK FINISHED, REFER LOG FILE FOR DETAILS.")
print("############################")
print("############################")

